In [1]:
import os
import glob
import numpy as np

In [18]:
b = np.load("dice.npy.npz")

In [21]:
print(b["loss"])

[[6.9505727e-01 1.3748487e-03 7.1242888e-04]
 [6.9039321e-01 8.2946470e-04 6.8835635e-04]]


In [2]:
path_img = "/home/user/Documents/dataset/Mindboggle101/mindboggle/image_in_MNI152_normalized"
path_seg = "/home/user/Documents/dataset/Mindboggle101/mindboggle/label_31_reID_merged"
img_file = []
seg_file = [] 
files = glob.glob(os.path.join(path_img,"*.nii.gz"))
for file in files:
    if "flipped" not in file:
        img_file.append(file)
        tmp = os.path.split(file)
        tmp = os.path.join(path_seg,tmp[1])
        seg_file.append(tmp)


In [9]:
out_path = "T7_Jdet100"
out_path = os.path.join(os.getcwd(),out_path)
print (out_path)

/home/user/Documents/DNVF/T7_Jdet100


In [6]:
filename = os.path.split(file)[1]
print(filename)

OASIS-TRT-20-15_flipped.nii.gz
